# Smart Logistic System in Walmart 

In [1]:
import numpy as np   #Numpy package would be the only package you could use for this project.

## Problem1: The Estimated Value for Each Warehouse
#### Note that you are not allowed to use dynamic programming in this problem!

In this problem 1, calculating the estimated total value and accumulated weight of each of 300 warehouses will be helpful for the later decision of cross-warehouse-transshipment solution in problem 2. 

### Step 1: Estimate the Maximized Total Value and Accumulated Weight of One Warehouse

Read the dataset from  "Products.csv" first.

In [2]:
file_name='Products.csv'
f = open(file_name,'r') #open file using file_name
lines = f.readlines() #read all lines from file

weight_list = []
value_list = []

for row in range(len(lines)):
    if((row+1) % 3 == 1):
        weight_list.append([eval(i) for i in lines[row].strip('\n').split(',')])
    elif((row+1) % 3 == 2):
        value_list.append([eval(i) for i in lines[row].strip('\n').split(',')])

#convert list to array
weights = np.array(weight_list)
values = np.array(value_list)

print(weights)
print(values)

[[112 100 108 ... 117  60  49]
 [ 96 105 117 ...  18  93  93]
 [115  95  93 ...  83  78  51]
 ...
 [ 35  91  66 ...  71  30 107]
 [ 44  84  21 ...  79  82  42]
 [ 97  47  19 ... 112  67  52]]
[[ 66.   170.66 130.   ... 119.31  23.    35.33]
 [110.69  68.57  67.5  ...  24.18  98.03 129.91]
 [169.25   8.86 102.68 ...  14.51 154.39  68.15]
 ...
 [  6.45  79.33  79.84 ...  83.18  50.49  36.6 ]
 [ 72.16 106.14  30.97 ...  99.78  92.69  57.08]
 [120.     5.91  22.29 ...  84.7   10.8   86.81]]


Now let's get the estimated value and weight for one warehouse.You need to write a function "get_max_value" which could do the following 4 things:
1. Calculate the Value_Weight Ratio for each product. 
2. Sort the calculated Value_Weight Ratios in descending order. 
3. Select the products from the top of the descending order to the bottom while looking at the accumulated         weight of products to make sure the accumulated weight does not exceed the warehouse weight capacity.
4. Calculate the  corresponding estimated value for this warehouse after finishing product selection.

Remember, take every product into consideration; otherwise, you may have the risk of missing products. You want to put as many product with highest Value_Weight Ratio as possible before reaching the weight capacity for this warehouse.

In [3]:
def get_max_value(weight,value,capacity):          #add capacity to the parentheses
    
    ratios = value/weight #calculate Value_Weight Ratio for each product
    
    product_list = [[weight[j],value[j],ratios[j]] for j in range(50)]
    product_list.sort(key=lambda x: x[2],reverse=True) #Sort the ratios in descending ordered
    
   
    current_weight = 0
    total_value = 0
     
    for k in range(len(product_list)):
        #Only append product when total weight is within limit.
        if(product_list[k][0] <= capacity - current_weight):
            current_weight += product_list[k][0]
            total_value += product_list[k][1]
    
    return round(total_value,2),int(current_weight)    
#Make sure you have calculated the total Value and accumulated Weight for One Warehouse
get_max_value(weights[0], values[0],650)

(1076.27, 650)

### Step 2: Estimate the Total Value for 300 Warehouses
Let's apply the same calculation process of step 1 for one warehouse to the 299 warehouses left. 

Keep in mind that calculating the estimated total value and accumulated weight of 300 warehouses will be helpful for the later decision of cross-warehouse-transshipment solution in problem 2. 

In [4]:
warehouse_value = [] #use a list to store each warehouse's total value and weight
weight_capacity = 650
''' we have 300 instances and one warehouse has 3 rows on the dataset. So, we will iterate for 900'''
for i in range(0,900,3):
    max_value = get_max_value(weights[int(i/3)],values[int(i/3)],weight_capacity)[0]
    total_weight = get_max_value(weights[int(i/3)],values[int(i/3)],weight_capacity)[1]
    warehouse_value.append([max_value,total_weight])
    

print(warehouse_value)         #show the warehouse_value you generated

[[1076.27, 650], [976.78, 648], [1231.7, 644], [1003.49, 649], [1099.97, 645], [1009.18, 650], [1196.45, 648], [975.12, 647], [1009.39, 648], [1054.0, 649], [948.03, 646], [1102.16, 648], [1133.12, 648], [1110.68, 645], [938.15, 644], [1059.3, 634], [1174.39, 650], [997.17, 649], [998.07, 641], [986.91, 642], [1221.07, 650], [1032.85, 649], [1046.92, 648], [1076.23, 649], [1090.61, 647], [1074.56, 650], [1050.91, 649], [1079.36, 649], [1040.33, 646], [1063.54, 647], [1036.14, 638], [1019.95, 650], [1127.27, 647], [1205.91, 646], [993.12, 647], [1078.94, 639], [1029.04, 650], [1221.12, 641], [1099.67, 649], [1114.26, 647], [1144.09, 650], [1007.43, 650], [1057.57, 649], [1158.73, 650], [1079.43, 647], [1120.63, 647], [1182.04, 649], [1059.25, 650], [1085.19, 650], [1088.89, 641], [1057.04, 650], [1225.87, 650], [915.62, 646], [1011.28, 641], [1039.84, 650], [1130.57, 649], [1013.58, 648], [1088.36, 650], [1095.93, 646], [1078.29, 636], [1058.07, 646], [1061.11, 650], [1034.35, 648], [11

## Problem 2: Top Alternative Selections

In this problem, you will simulate the cross-warehouse-transshipment solutions when a random warehouse is out-of-stock during the promotion period.

### Step 1: Generate the 300*300 distance matrix and write the results to "Distances.csv"
Let's generate a distance matrix among the 300 warehouses first. Each of the distances can be generated by using a normal distribution with a mean of 500 and a standard deviation of 300. 

Be careful, all the distances generated should be positive numbers and should be rounded to integers using the round() function. After successfully generating the distance matrix, please write it to a new CSV file called “Distances.csv”.

In [5]:
# Generate the Distance matrix 300*300

up = np.random.normal(500, 300, (300,300))

for i in range(300):    #generate a 300*300 upper triangle matrix
    for j in range(300):
        if i >= j:      #change numbers to 0 if it is not in the upper triangle
            up[i][j] = 0
        else:
            if up[i][j] < 0:
                up[i][j] = abs(up[i][j]) #change negative numbers into positive
            up[i][j] = round(up[i][j])   #round numbers to integers
            
low = up.T #transpose the upper traingle matrix to get lower triangle matrix
distances = low + up #Add the lower triangle matrix to the upper triangle matrix to get the symmetrical distances matrix

    
distances            #show the distance matrix you generated 

# Check your maxtrix and see if all numbers are positive and rounded to integer

array([[   0.,  360.,  400., ...,  596.,  534.,  931.],
       [ 360.,    0.,  216., ...,   37.,  542.,  725.],
       [ 400.,  216.,    0., ...,  628.,  787.,  318.],
       ...,
       [ 596.,   37.,  628., ...,    0.,  594., 1041.],
       [ 534.,  542.,  787., ...,  594.,    0.,   31.],
       [ 931.,  725.,  318., ..., 1041.,   31.,    0.]])

Now we are going to save the file in 'Distance.csv'

In [6]:
f = open("Distances.csv", "w") # We will create this file.

for i in range(len(distances)):
    for j in distances[i]:
         f.write(str(j) + ',')   
    f.write("\n") #change to a new line when we finish writing one row
       
f.close()

### Step2：Select the Alternative Warehouse ("Helper")

Marlon randomly selects No.p warehouse that is out-of-stock in this simulation.

In [7]:
'''p is offered by Malron. Do not change the code below.'''

p=np.random.randint(0,300) 
p

176

There are 3 things to do:
1. Based on the generated p, find the corresponding distance between this warehouse No.p and each of the other Helpers from the distance matrix you generated in Step 1 of Problem 2.
2. Find the corresponding total value and total weight of all products stored in each Helper (you have calculated these numbers in Problem 1).
3. Calculate the “Value_per_Weight” ratio for each Helper, sort the ratio in descending order and choose top helpers with 10 highest “Value_per_Weight”ratios.

###  $Value\_per\_Weight = \frac{Total\_Value}{Total\_Weight}-Distance\times\ Transpotation\_Cost$

#### Note that you are not allowed to use dynamic programming in this problem!

In [8]:
indexes = list(range(300)) #Generate the index list with continuous number from 0 to 299 to indicate each warehouse
indexes.remove(p) #Remove the index of the provided no.p warehouse
cost = 0.015 #Speicify the given transportation cost

#Set a list to store warehouse index and value_per_weight
vw_ratio = [[i,warehouse_value[i][0]/warehouse_value[i][1]-distances[p][i]*cost] for i in indexes]
vw_ratio.sort(key=lambda x: x[1], reverse=True) #Sort the value_per_weight of helpers from high to low
vw_ratio[:10] #Show only the 10 highest v_p_w  
    
    
# Show the cross-warehouse-transshipment solution when No.p warehouse is out-of-stock during the promotion period. 
# The solution should contain Helper indexes and the corresponding "Value_Per_Weight" ratios. 

[[4, 1.4203798449612404],
 [123, 1.321630602782071],
 [63, 1.2217283950617284],
 [28, 1.1904179566563466],
 [217, 1.1750619195046441],
 [201, 1.1552255054432348],
 [37, 1.0950234009360371],
 [151, 0.7854391371340523],
 [84, 0.7560493827160495],
 [221, 0.7504866562009419]]

#### Note: Due to random assignment of No.p warehouse, it's acceptable if your recommended solution changes each time you re-run the code. 